## Setup the Notebook:

In [1]:
#####################################
#
#   Load libraries
#
#####################################

from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML

# Project-specific Libraries
import numpy as np
import pandas as pd
from pathlib import Path
import pyhere
import os
import rfmix_reader

# Statistical Analysis Libraries
from scipy import stats
from scipy.stats import shapiro, boxcox, zscore, spearmanr
import statsmodels.api as sm

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#####################################
#
#   Set Screen Attributes
#
#####################################

InteractiveShell.ast_node_interactivity = "all"

# set cell width
display(HTML("<style>.container { width:100% !important; }</style>"))

# set cell output window height
display(HTML("<style>div.output_scroll { height: 160em;} </style>"))

RuntimeError: Failed to dlopen libcudart.so.12

In [3]:
print(os.getcwd())
print(pyhere.here())
print(Path(pyhere.here()))

/gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/visualization/bivariate_tests/_h
/gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking
/gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking


## Load the Data:

### Groundtruth data:

In [4]:
def load_groundtruth_data(gt_dir: str) -> pd.DataFrame:
    gt_dir = pyhere.here(gt_dir)
    all_dfs = []

    for chrom in range(1, 23):  # chromosomes 1–22
        fname = gt_dir / f"global_ancestry_chr{chrom}.tsv"
        if fname.exists():
            df = pd.read_csv(fname, sep="\t", header=0)

            # Rename ancestry columns
            df = df.rename(columns={"CEU": "EUR", "YRI": "AFR"})

            # Add chromosome suffix to ancestry columns
            ancestry_cols = [col for col in df.columns if col not in ["Sample"]]
            df = df.rename(columns={col: f"{col}_chr{chrom}" for col in ancestry_cols})

            all_dfs.append(df)
        else:
            print(f"Missing file: {fname}")

    if not all_dfs:
        raise FileNotFoundError(f"No global_ancestry_chr*.tsv files found in {gt_dir}")

    # Merge across chromosomes on 'Sample'
    merged_df = all_dfs[0]
    for df in all_dfs[1:]:
        merged_df = merged_df.merge(df, on="Sample", how="outer")
        
    merged_df = merged_df.sort_values(by='Sample')
    
    # Extract numeric part of Sample for sorting
    merged_df["Sample_num"] = merged_df["Sample"].str.extract(r"(\d+)").astype(int)
    merged_df = merged_df.sort_values(by="Sample_num").drop(columns="Sample_num")
    

    return merged_df

In [7]:
gt_data = load_groundtruth_data("input/simulations/three_populations/ground_truth/_m/")

gt_data

Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m/global_ancestry_chr1.tsv
Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m/global_ancestry_chr2.tsv
Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m/global_ancestry_chr3.tsv
Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m/global_ancestry_chr4.tsv
Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m/global_ancestry_chr5.tsv
Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m/global_ancestry_chr6.tsv
Missing file: /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulat

FileNotFoundError: No global_ancestry_chr*.tsv files found in /gpfs/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/ground_truth/_m

### RFMix Data:

In [6]:
rfmix_path = ("/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/three_populations/_m/rfmix-out/")

loci, rf_q, admix = rfmix_reader.read_rfmix(file_prefix = rfmix_path,
                                            verbose = True)

GPU 0: NVIDIA A100-PCIE-40GB
  Total memory: 39.49 GB
  CUDA capability: 8.0
Multiple RFMIX file sets read in this order: ['chr15', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']


Mapping loci information:   0%|                                                                                         | 0/7 [00:00<?, ?it/s]

FileNotFoundError: File /projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/two_populations/_m/rfmix-out/chr15.fb.tsv not found.

In [31]:
def load_rfmix_data(rf_q: pd.DataFrame) -> pd.DataFrame:
    # Rename sample_id to Sample
    df = rf_q.rename(columns={"sample_id": "Sample"}).copy()

    # Append chrom to AFR and EUR column names
    df["AFR_chrom"] = df.apply(lambda row: row["AFR"], axis=1)
    df["EUR_chrom"] = df.apply(lambda row: row["EUR"], axis=1)
    df["AFR"] = df["AFR"]
    df["EUR"] = df["EUR"]
    
    # Pivot to wide format
    df_wide = df.pivot(index="Sample", columns="chrom", values=["AFR", "EUR"])

    # Flatten MultiIndex columns
    df_wide.columns = [f"{ancestry}_{chrom}" for ancestry, chrom in df_wide.columns]

    df_wide = df_wide.reset_index()

    # Ensure natural numeric sorting of Sample if numeric IDs inside
    if df_wide["Sample"].str.extract(r"(\d+)").notnull().all().any():
        df_wide["Sample_num"] = df_wide["Sample"].str.extract(r"(\d+)").astype(float)
        df_wide = df_wide.sort_values(by="Sample_num").drop(columns="Sample_num")
    else:
        df_wide = df_wide.sort_values(by="Sample")

    # Ensure columns are ordered: Sample first, then AFR_chr1, EUR_chr1, ...
    chroms = df["chrom"].unique()
    # Sort chromosomes numerically if possible
    try:
        chroms_sorted = sorted(chroms, key=lambda x: int(str(x).replace("chr", "")))
    except:
        chroms_sorted = sorted(chroms)

    cols = ["Sample"]
    for c in chroms_sorted:
        if f"AFR_{c}" in df_wide.columns and f"EUR_{c}" in df_wide.columns:
            cols.extend([f"AFR_{c}", f"EUR_{c}"])
    df_wide = df_wide.reindex(columns=cols)

    return df_wide

In [33]:
rf_data = load_rfmix_data(rf_q)

rf_data

,Sample,AFR_chr7,EUR_chr7,AFR_chr8,EUR_chr8,AFR_chr9,EUR_chr9,AFR_chr10,EUR_chr10,AFR_chr11,...,AFR_chr18,EUR_chr18,AFR_chr19,EUR_chr19,AFR_chr20,EUR_chr20,AFR_chr21,EUR_chr21,AFR_chr22,EUR_chr22
0,Sample_1,0.68016,0.31984,0.88720,0.11280,0.84473,0.15527,0.83607,0.16393,0.86843,...,0.55192,0.44808,0.57509,0.42491,0.85382,0.14618,1.00000,0.00000,0.78239,0.21761
111,Sample_2,0.84461,0.15539,0.88359,0.11641,0.74885,0.25115,0.63890,0.36110,0.88716,...,0.92147,0.07853,1.00000,0.00000,0.68933,0.31067,0.80204,0.19796,0.57705,0.42295
222,Sample_3,0.63963,0.36037,0.87610,0.12390,0.65033,0.34967,0.96971,0.03029,0.58883,...,0.88446,0.11554,0.70295,0.29705,1.00000,0.00000,0.85389,0.14611,0.98714,0.01286
333,Sample_4,0.85708,0.14292,0.82899,0.17101,0.68704,0.31296,0.61517,0.38483,0.86314,...,0.87024,0.12976,0.81376,0.18624,0.86754,0.13246,0.76187,0.23813,0.41601,0.58399
444,Sample_5,0.83696,0.16304,0.90790,0.09210,0.93143,0.06857,0.92634,0.07366,0.61614,...,0.90747,0.09253,0.88675,0.11325,0.68279,0.31721,0.61167,0.38833,0.53434,0.46566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Sample_496,0.60971,0.39029,0.92987,0.07013,0.88394,0.11606,0.98899,0.01101,0.90683,...,0.75228,0.24772,0.94253,0.05747,0.71011,0.28989,0.40611,0.59389,0.82322,0.17678
441,Sample_497,0.92406,0.07594,0.63708,0.36292,0.84641,0.15359,0.92878,0.07122,0.86518,...,0.51795,0.48205,0.80711,0.19289,0.92478,0.07522,0.81444,0.18556,0.73456,0.26544
442,Sample_498,0.74099,0.25901,0.91183,0.08817,0.88985,0.11015,0.68259,0.31741,0.86079,...,0.83475,0.16525,0.79602,0.20398,0.79797,0.20203,1.00000,0.00000,1.00000,0.00000
443,Sample_499,0.59494,0.40506,0.70152,0.29848,0.77689,0.22311,0.40950,0.59050,0.83661,...,0.73133,0.26867,0.96613,0.03387,0.80328,0.19672,0.73146,0.26854,0.87362,0.12638


### Flare Data: 

In [34]:
help(rfmix_reader.read_flare)

Help on function read_flare in module rfmix_reader._read_flare:

read_flare(file_prefix: str, chunk_size: numpy.int32 = 1000000, verbose: bool = True) -> Tuple[pandas.core.frame.DataFrame, pandas.core.frame.DataFrame, dask.array.core.Array]
    Read Flare files into data frames and a Dask array.

    Parameters
    ----------
    file_prefix : str
        Path prefix to the set of Flare files. It will load all of the chromosomes
        at once.
    chunk_size : int
        Number of records to read per chunk.
    verbose : bool, optional
        :const:`True` for progress information; :const:`False` otherwise.
        Default:`True`.

    Returns
    -------
    loci : :class:`DataFrame`
        Loci information for the FB data.
    g_anc : :class:`DataFrame`
        Global ancestry by chromosome from Flare.
    admix : :class:`dask.array.Array`
        Local ancestry per population stacked (variants, samples, ancestries).
        This is in alphabetical order of the populations. This

In [5]:
flare_path = ("/projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/two_populations/_m/flare-out/")

loci, g_anc, admix = rfmix_reader.read_flare(file_prefix = flare_path,
                                             verbose = True)

GPU 0: NVIDIA A100-PCIE-40GB
  Total memory: 39.49 GB
  CUDA capability: 8.0


FileNotFoundError: No valid FLARE files found for prefix: /projects/p32505/users/manuel/rfmix_reader-benchmarking/input/simulations/two_populations/_m/flare-out/

In [ ]:
g_anc

In [ ]:
def load_flare_data(g_anc: pd.DataFrame) -> pd.DataFrame:
    # Rename sample_id to Sample
    df = g_anc.rename(columns={"sample_id": "Sample"}).copy()

    # Append chrom to AFR and EUR column names
    df["AFR_chrom"] = df.apply(lambda row: row["AFR"], axis=1)
    df["EUR_chrom"] = df.apply(lambda row: row["EUR"], axis=1)
    df["AFR"] = df["AFR"]
    df["EUR"] = df["EUR"]
    
    # Pivot to wide format
    df_wide = df.pivot(index="Sample", columns="chrom", values=["AFR", "EUR"])

    # Flatten MultiIndex columns
    df_wide.columns = [f"{ancestry}_{chrom}" for ancestry, chrom in df_wide.columns]

    df_wide = df_wide.reset_index()

    # Ensure natural numeric sorting of Sample if numeric IDs inside
    if df_wide["Sample"].str.extract(r"(\d+)").notnull().all().any():
        df_wide["Sample_num"] = df_wide["Sample"].str.extract(r"(\d+)").astype(float)
        df_wide = df_wide.sort_values(by="Sample_num").drop(columns="Sample_num")
    else:
        df_wide = df_wide.sort_values(by="Sample")

    # Ensure columns are ordered: Sample first, then AFR_chr1, EUR_chr1, ...
    chroms = df["chrom"].unique()
    # Sort chromosomes numerically if possible
    try:
        chroms_sorted = sorted(chroms, key=lambda x: int(str(x).replace("chr", "")))
    except:
        chroms_sorted = sorted(chroms)

    cols = ["Sample"]
    for c in chroms_sorted:
        if f"AFR_{c}" in df_wide.columns and f"EUR_{c}" in df_wide.columns:
            cols.extend([f"AFR_{c}", f"EUR_{c}"])
    df_wide = df_wide.reindex(columns=cols)

    return df_wide

In [ ]:
flare_data = load_flare_data(g_anc)

flare_data

## Data Analysis:

### Test for Normality: Shapiro-Wilk Test for Normality and KS Test for Normality.
- Groundtruth data DOES NOT follow a normal distribution 
- RFMix data DOES NOT follow a normal distribution

In [47]:
# Perform the Shapiro-Wilk test for normality
def shapirowilk_norm_test(df, alpha=0.05): 
    
    # Drop the sample ID column if present
    df = df.drop(columns=['Sample'], errors='ignore')
    
    for col in df.columns:
        data = df[col].dropna()
        w_stat, p_value = shapiro(data)
        print(f"'{col}': W statistic = {w_stat:.4}, p-value = {p_value:.4}")
        if p_value < alpha:
            print(f"The Shapiro-Wilk test suggests a significant departure from normality.")
        else:
            print(f"The Shapiro-Wilk test suggests that the data is following a normal distribution.")

In [53]:
# Conduct the Kolmogorov-Smirnov test for normality
def ks_norm_test(df, alpha=0.05): 
    
    # Drop the sample ID column if present
    df = df.drop(columns=['Sample'], errors='ignore')
    
    for col in df.columns:
        data = df[col].dropna()
        ks_statistic, p_value = stats.kstest(data, 'norm')
        print(f"'{col}': K-S statistic = {ks_statistic:.4}, p-value = {p_value:.4}")
        if p_value < alpha:
            print(f"The Kolmogorov-Smirnov test suggests a significant departure from normality.")
        else:
            print(f"The Kolmogorov-Smirnov test suggests that the data is following a normal distribution.")

In [52]:
shapirowilk_norm_test(rf_data)

'AFR_chr7': W statistic = 0.9701, p-value = 1.429e-08
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr7': W statistic = 0.9701, p-value = 1.429e-08
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr8': W statistic = 0.958, p-value = 9.668e-11
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr8': W statistic = 0.958, p-value = 9.668e-11
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr9': W statistic = 0.9587, p-value = 1.27e-10
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr9': W statistic = 0.9587, p-value = 1.27e-10
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr10': W statistic = 0.9686, p-value = 7.134e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr10': W statistic = 0.9686, p-value = 7.134e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'A

In [54]:
shapirowilk_norm_test(gt_data)

'EUR_chr1': W statistic = 0.9721, p-value = 3.642e-08
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr1': W statistic = 0.9721, p-value = 3.642e-08
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr2': W statistic = 0.9653, p-value = 1.756e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr2': W statistic = 0.9653, p-value = 1.756e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr3': W statistic = 0.9642, p-value = 1.125e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr3': W statistic = 0.9642, p-value = 1.125e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'EUR_chr4': W statistic = 0.9662, p-value = 2.572e-09
The Shapiro-Wilk test suggests a significant departure from normality.
'AFR_chr4': W statistic = 0.9662, p-value = 2.572e-09
The Shapiro-Wilk test suggests a significant departure from normality.


In [55]:
ks_norm_test(rf_data)

'AFR_chr7': K-S statistic = 0.6789, p-value = 7.429e-229
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr7': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr8': K-S statistic = 0.6696, p-value = 1.369e-221
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr8': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr9': K-S statistic = 0.6744, p-value = 2.277e-225
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr9': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr10': K-S statistic = 0.6891, p-value = 4.739e-237
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr10': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-

In [56]:
ks_norm_test(gt_data)

'EUR_chr1': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr1': K-S statistic = 0.6889, p-value = 6.415e-237
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr2': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr2': K-S statistic = 0.6733, p-value = 1.751e-224
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr3': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr3': K-S statistic = 0.6683, p-value = 1.185e-220
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'EUR_chr4': K-S statistic = 0.5, p-value = 4.113e-116
The Kolmogorov-Smirnov test suggests a significant departure from normality.
'AFR_chr4': K-S statistic = 0.6722, p-value = 1.27e-223
The Kolmogorov-Smi

### Linearity and Homoscedascity:

In [ ]:
def analyze_relationship(x, y, x_label="X", y_label="Y"):
    """
    Plot scatterplot with regression line and residuals vs fitted values
    to assess linearity and homoscedasticity.
    """

    # Put into dataframe
    df = pd.DataFrame({x_label: x, y_label: y}).dropna()

    # === (2) Scatterplot with regression line ===
    plt.figure(figsize=(6, 5))
    sns.regplot(data=df, x=x_label, y=y_label, line_kws={"color": "red"})
    plt.title("Scatterplot with regression line")
    plt.show()

    # === Fit linear regression ===
    X = sm.add_constant(df[x_label])  # add intercept
    model = sm.OLS(df[y_label], X).fit()
    df["fitted"] = model.fittedvalues
    df["residuals"] = model.resid

    # === (3) Residuals vs fitted values ===
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=df["fitted"], y=df["residuals"])
    plt.axhline(0, color="red", linestyle="--")
    plt.title("Residuals vs Fitted")
    plt.xlabel("Fitted values")
    plt.ylabel("Residuals")
    plt.show()

    return model.summary()


### BoxPlots for Outlier Analysis:

In [ ]:
# Distribution plots for the dataframe        
def box_plots(df):
    for column in df.columns:
        df = df.drop(columns=['Sample'], errors='ignore')
        data = df[column].dropna()
        plt.figure(figsize=(4, 3))
        sns.boxplot(data, color="skyblue", fill=False, width=.5, notch=True, showcaps=True)
        plt.title(f"Boxplot of {column}")
        plt.xlabel(column)
        plt.ylabel("Frequency")
        plt.show()

### Spearman Correlation:

In [57]:
def spearman_corr_table(df):
    
    df = df.drop(columns=['Sample'], errors='ignore')
    cols = df.columns
    n = len(cols)
    corr_matrix = pd.DataFrame('', index=cols, columns=cols)

    for i in range(n):
        for j in range(n):
            if i == j:
                corr_matrix.iloc[i, j] = '1.000'
            elif i > j:
                corr, p = spearmanr(df[cols[i]], df[cols[j]])
                if p < 0.001:
                    stars = '***'
                elif p < 0.01:
                    stars = '**'
                elif p < 0.05:
                    stars = '*'
                else:
                    stars = ''
                corr_str = f"{corr:.3}{stars}"
                corr_matrix.iloc[i, j] = corr_str
            else:
                corr_matrix.iloc[i, j] = ''  # Blank upper triangle
    return corr_matrix

In [58]:
spearman_corr_table(rf_data)

,AFR_chr7,EUR_chr7,AFR_chr8,EUR_chr8,AFR_chr9,EUR_chr9,AFR_chr10,EUR_chr10,AFR_chr11,EUR_chr11,...,AFR_chr18,EUR_chr18,AFR_chr19,EUR_chr19,AFR_chr20,EUR_chr20,AFR_chr21,EUR_chr21,AFR_chr22,EUR_chr22
AFR_chr7,1.000,,,,,,,,,,...,,,,,,,,,,
EUR_chr7,-1.0***,1.000,,,,,,,,,...,,,,,,,,,,
AFR_chr8,-0.0442,0.0442,1.000,,,,,,,,...,,,,,,,,,,
EUR_chr8,0.0442,-0.0442,-1.0***,1.000,,,,,,,...,,,,,,,,,,
AFR_chr9,0.038,-0.038,-0.0409,0.0409,1.000,,,,,,...,,,,,,,,,,
EUR_chr9,-0.038,0.038,0.0409,-0.0409,-1.0***,1.000,,,,,...,,,,,,,,,,
AFR_chr10,-0.0772,0.0772,0.00971,-0.00971,0.0352,-0.0352,1.000,,,,...,,,,,,,,,,
EUR_chr10,0.0772,-0.0772,-0.00971,0.00971,-0.0352,0.0352,-1.0***,1.000,,,...,,,,,,,,,,
AFR_chr11,0.0233,-0.0233,-0.0457,0.0457,-0.0324,0.0324,0.046,-0.046,1.000,,...,,,,,,,,,,
EUR_chr11,-0.0233,0.0233,0.0457,-0.0457,0.0324,-0.0324,-0.046,0.046,-1.0***,1.000,...,,,,,,,,,,


In [59]:
spearman_corr_table(gt_data)

,EUR_chr1,AFR_chr1,EUR_chr2,AFR_chr2,EUR_chr3,AFR_chr3,EUR_chr4,AFR_chr4,EUR_chr5,AFR_chr5,...,EUR_chr18,AFR_chr18,EUR_chr19,AFR_chr19,EUR_chr20,AFR_chr20,EUR_chr21,AFR_chr21,EUR_chr22,AFR_chr22
EUR_chr1,1.000,,,,,,,,,,...,,,,,,,,,,
AFR_chr1,-1.0***,1.000,,,,,,,,,...,,,,,,,,,,
EUR_chr2,-0.0349,0.0349,1.000,,,,,,,,...,,,,,,,,,,
AFR_chr2,0.0349,-0.0349,-1.0***,1.000,,,,,,,...,,,,,,,,,,
EUR_chr3,0.0426,-0.0426,0.009,-0.009,1.000,,,,,,...,,,,,,,,,,
AFR_chr3,-0.0426,0.0426,-0.009,0.009,-1.0***,1.000,,,,,...,,,,,,,,,,
EUR_chr4,-0.00399,0.00399,-0.0122,0.0122,-0.0467,0.0467,1.000,,,,...,,,,,,,,,,
AFR_chr4,0.00399,-0.00399,0.0122,-0.0122,0.0467,-0.0467,-1.0***,1.000,,,...,,,,,,,,,,
EUR_chr5,0.035,-0.035,0.104*,-0.104*,-0.00402,0.00402,0.00507,-0.00507,1.000,,...,,,,,,,,,,
AFR_chr5,-0.035,0.035,-0.104*,0.104*,0.00402,-0.00402,-0.00507,0.00507,-1.0***,1.000,...,,,,,,,,,,
